# RL Circuit
Practical Electronics pg 142

## RL Energising Schematic
$V_R$ = Voltage over the resistor

$V_L$ = Voltage over the inductor

$I$ = Current through both.

In [73]:
# RL Energising Schematic
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import schemdraw
import schemdraw.elements as elm

with schemdraw.Drawing() as d:
    mc = 'blue'
    s = elm.Switch(action='close').label('$t=0$').dot()
    d.push()
    elm.Line().dot(open=True).color(mc)
    elm.Gap().down().label(['+','$V_R$','-']).dot(open=True).color(mc)
    elm.Gap().label(['+','$V_L$','-']).dot(open=True).color(mc)
    elm.Line().left().color(mc)
    d.pop()
    r = elm.Resistor().down().label('$R$\n100Ω', loc='bot').dot()
    elm.CurrentLabel(top=False, ofst=0.5).at(r).label('$I$')
    d.push()
    elm.Line().right().color(mc)
    d.pop()
    elm.Inductor().label('$L$\n20mH', loc='bottom').dot()
    elm.Line().tox(s.start)
    #elm.Ground()
    elm.SourceV().toy(s.start).label('$V_S$\n10V')

## Energising an RL circuit.
Voltage and Current flow over Time for DC voltage.

$ I = \frac{V_S}{R}(1-e^{-t/(\frac{L}{R})}) $

$ V_{R} = IR = V_{S}(1-e^{-t/(\frac{L}{R})}) $

$ V_{L} = L\frac{dI}{dt} = V_{S}e^{-t/(\frac{L}{R})} $

$ \tau = \frac{L}{R} $ time constant

Notice at ${t} \geq 5 \tau$ (or ${t} \geq 5\frac{L}{R}$):
* $\frac{dI}{dt} \approx 0$ A/s - Current is not changing.
* ${V_L} \approx 0$ V - Inductor has reached zero impedance.
* ${I} \approx \frac{V_S}{R}$ - Current has reach it's maximum.

Notice at ${t} = 0$:
* $V_L$ instantly goes to $V_S$. Assuming ideal components.
* The inductors impedance is high because the rate of current change, $\frac{dI}{dt}$, is at its highest.

Notice:
* Time period $5\tau$ (or $5\frac{L}{R}$) is relatively short. E.g. milli-seconds compared to RC circut (where $\tau = {RC}$)
* Current is relatively high compared to RC circuit because a small $R$ is needed to increase $\tau$.
* Increasing $R$ shortens $\tau$ and limits the current $I$. The later, is just Ohms law.

In [84]:
# RL Energising Voltage and Current over Time
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

def rl_plot(Vs=10, R_ohms=100, L_mH=20, Duration_ms=1):
    L = L_mH/1000 # mH -> H
    R = R_ohms
    time_ms = np.linspace(start=0, stop=Duration_ms, num=100)
    t = time_ms/1000 # ms -> secs
    if (R==0):
        R=1
    tc = L/R
    Vr = Vs * (1 - np.exp(-t / tc))
    Vl = Vs * np.exp(-t / tc)
    I = Vr/R*1000 # A -> mA

    fig, vp = plt.subplots(figsize=(8, 6))
    vp.set_xlabel('Time (ms)')
    vp.set_ylabel('Voltage (V)')
    vp.set_ylim((0,Vs))
    vp.plot(time_ms, Vr, label='$V_R$')
    vp.plot(time_ms, Vl, label='$V_L$')
    vp.grid(True)
    vp.legend(loc='upper left')
    vp.set_title("Energising an RL Circuit\n5$\\tau$ = 5($L/R$) = {tc:.1f}ms".format(tc=tc*5*1000))

    ip = vp.twinx()
    color = 'tab:green'
    ip.set_ylabel('Current (mA)', color=color)
    ip.set_ylim((0,200))
    ip.plot(time_ms, I, label='$I$', color=color)
    ip.tick_params(axis='y', labelcolor=color)
    ip.legend(loc='lower right')
    plt.show()

interact(rl_plot, Vs=(1, 20, 1.0), R_ohms=(50, 500, 50), L_mH=(10, 1000, 10), Duration_ms=(1, 60, 1))

interactive(children=(FloatSlider(value=10.0, description='Vs', max=20.0, min=1.0, step=1.0), IntSlider(value=…

<function __main__.rl_plot(Vs=10, R_ohms=100, L_mH=20, Duration_ms=1)>

## RL Circuit Deenergising Schematic
* ${t} = 0$, Switch opened.
* Rapid collapse of the inductors magnetic field. Energy stored in the magnetic field returns to the circuit.
* Induces a voltage many times larger than the applied voltage. Voltage proportional to rate of the decreasing field.
* Commonly causes sparks or arcs to form at the switch contacts.
* Large inductance produces large current and large amounts of energy released in a short period of time.
* Switch contact may burn or melt.
* RC Snubber network across the switch contacts supresses the spark or arcing. 


In [ ]:
# RL Circuit Deenergising Snubber Network
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import schemdraw
import schemdraw.elements as elm

with schemdraw.Drawing() as d:
    mc = 'blue'
    l1 = elm.Line(unit=1).dot()
    d.push()
    elm.Line(unit=1).down()
    r = elm.Resistor(unit=1.5).right()
    c = elm.Capacitor(unit=1)
    d.push()
    snubber = elm.Encircle([r,c],pady=0.3).linestyle('--').color('red')
    elm.Annotate().at(snubber.S).delta(dy=-0.5).label('Snubber').color('red')
    d.pop()
    l2 = elm.Line(unit=1).up().dot()
    d.pop()
    s = elm.Switch(action='open').label('$t=0$\nSpark Discharge').tox(l2.end)
    elm.Line(unit=1)
    elm.Resistor(unit=1.8).down().label('R', loc='bot')
    elm.Inductor(unit=1.8).label('L').label('Large\nInductance', loc='bottom')
    elm.Line().tox(l1.start)
    elm.SourceV().toy(l1.start).label('V$_S$')

### Discharge to Ground
A theoretical switch with no break in circuit, allows the current and voltage expressions for a deenergising RL circuit to be derived.

In [86]:
# RL Circuit Deenergising - To ground switch
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import schemdraw
import schemdraw.elements as elm

with schemdraw.Drawing() as d:
    mc = 'blue'
    b = elm.Line().right()
    s = elm.SwitchSpdt2(action='open').left().anchor('c').label('$t=0$')
    elm.Line().right().dot().at(s.a)
    d.push()
    elm.Line().dot(open=True).color(mc)
    elm.Gap().down().label(['+','$V_R$','-']).dot(open=True).color(mc)
    elm.Gap().label(['-','$V_L$','+']).dot(open=True).color(mc)
    elm.Line().left().color(mc)
    d.pop()
    r = elm.Resistor().down().label('R\n100Ω', loc='bot').dot()
    elm.CurrentLabel(top=False, ofst=0.5).at(r).label('$I$')
    d.push()
    elm.Line().right().color(mc)
    d.pop()
    elm.Inductor().label('L\n20$\\mu$H')
    elm.Line().left().tox(b.start)
    v = elm.SourceV().toy(b.start).label('V$_S$\n10V')
    elm.Line().at(s.b).toy(v.start).dot()

## Deenergising an RL circuit.
Voltage and Current flow over Time.

$ I = \frac{V_S}{R}e^{-t/(\frac{L}{R})} $

$ V_{R} = IR = V_{S}e^{-t/(\frac{L}{R})} $

$ V_{L} = L\frac{dI}{dt} = -V_{S}e^{-t/(\frac{L}{R})} $

$ \tau = \frac{L}{R} $ time constant

Notice at ${t} = 0$:
* The circuit current instantly tries to go from $I$ to 0 A.
* The instantaneous current rate change is $\frac{dI}{dt} \approx \infty$ A/s. So ${V_L} = L\frac{dI}{dt}$ can be very high in the real world.
* $V_L$ instantly goes from 0 V to -$V_S$. Assuming ideal components.
* The inductor acts as a current source as it's magnetic field collapses. This holds up $V_R$.
* $V_R$ stays at $V_S$ at ${t} = 0$, but immediately starts to decline as the induced current declines.

Notice at ${t} \geq 5 \tau$:
* $V_R \approx 0$ V, $V_L \approx 0$ V, and $I \approx 0$ A
* $\frac{dI}{dt} \approx 0$ A/s

In [98]:
# RL Deenergising Voltage and Current over Time
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

def rl_plot(Vs=10, R_ohms=100, L_mH=20, Duration_ms=1):
    L = L_mH/1000 # mH -> H
    R = R_ohms
    time_ms = np.linspace(start=0, stop=Duration_ms, num=100)
    t = time_ms/1000 # ms -> secs
    if (R==0):
        R=1
    tc = L/R
    Vr = Vs * np.exp(-t / tc)
    Vl = -Vr
    I = Vr/R*1000 # A -> mA

    fig, vp = plt.subplots(figsize=(8, 6))
    vp.set_xlabel('Time (ms)')
    vp.set_ylabel('Voltage (V)')
    vp.set_ylim((-Vs,Vs))
    vp.plot(time_ms, Vr, label='$V_R$')
    vp.plot(time_ms, Vl, label='$V_L$')
    vp.grid(True)
    vp.legend(loc='upper left')
    vp.set_title("Deenergising an RL Circuit\n5$\\tau$ = 5($L/R$) = {tc:.1f}ms".format(tc=tc*5*1000))

    ip = vp.twinx()
    color = 'tab:green'
    ip.set_ylabel('Current (mA)', color=color)
    ip.set_ylim((-200,200))
    ip.plot(time_ms, I, label='$I$', color=color)
    ip.tick_params(axis='y', labelcolor=color)
    ip.legend(loc='lower right')
    plt.show()

interact(rl_plot, Vs=(1, 20, 1.0), R_ohms=(50, 500, 50), L_mH=(10, 1000, 10), Duration_ms=(1, 60, 1))

interactive(children=(FloatSlider(value=10.0, description='Vs', max=20.0, min=1.0, step=1.0), IntSlider(value=…

<function __main__.rl_plot(Vs=10, R_ohms=100, L_mH=20, Duration_ms=1)>